## Data preprocessing

Preprocesses classification and weather data.
- Filter the right columns
- Change columns to datetime
- Make dummy variable
- Make rain and temperature variable from the forecast of a week ago
- merge dataframes

In [149]:
# Import libraries
import pandas as pd
import datetime

## Assess data

In [150]:
# Load data and show first 5 rows
df_classification = pd.read_excel('data/classificatie_OVV_ZVT.xlsx')
df_weather = pd.read_excel('data/openWeather20210705_20211001.xlsx')
print('First 5 rows of the classification dataframe:')
print(df_classification.head())
print('---------------------------')
print('First five rows of the weather dataframe:')
print(df_weather.head())

First 5 rows of the classification dataframe:
   ritnummer verkeersdatum_ams  vertrekstationUicCode  aankomststationUicCode  \
0       5416        2021-07-05                8400733                 8400501   
1       5441        2021-07-05                8400501                 8400733   
2       5447        2021-07-05                8400501                 8400733   
3       5448        2021-07-05                8400733                 8400501   
4       5450        2021-07-05                8400733                 8400501   

  aankomststationVerkorting vertrekstationVerkorting  classificatie  \
0                       OVN                      ZVT              1   
1                       ZVT                      OVN              1   
2                       ZVT                      OVN              1   
3                       OVN                      ZVT              1   
4                       OVN                      ZVT              1   

         materieeltype             vertr

### Filter data

In [151]:
## Filter data
df_classification_filter = df_classification[['verkeersdatum_ams', 'aankomststationVerkorting', 'vertrekstationVerkorting', 'classificatie', 'vertrekmoment_utc']]
df_weather_filter = df_weather[['forecast dt iso', 'slice dt iso', 'temperature', 'rain']]

## Feature engineering

In [152]:
# Change objects to datetime
df_classification_filter['vertrekmoment_utc'] = pd.to_datetime(df_classification_filter['vertrekmoment_utc'])
df_weather_filter['forecast dt iso'] = pd.to_datetime(df_weather_filter['forecast dt iso'], format='%Y-%m-%d %H:%M:%S %z UTC')
df_weather_filter['slice dt iso'] = pd.to_datetime(df_weather_filter['slice dt iso'], format='%Y-%m-%d %H:%M:%S %z UTC')

C:\Users\marieke.beerepoo_nsp\AppData\Local\Temp\ipykernel_11072\1287926287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification_filter['vertrekmoment_utc'] = pd.to_datetime(df_classification_filter['vertrekmoment_utc'])
C:\Users\marieke.beerepoo_nsp\AppData\Local\Temp\ipykernel_11072\1287926287.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather_filter['forecast dt iso'] = pd.to_datetime(df_weather_filter['forecast dt iso'], format='%Y-%m-%d %H:%M:%S %z UTC')
C:\Users\marieke.be

In [153]:
# Define rush hour times
start_morning_rush = datetime.time(8, 0, 0)
end_morning_rush = datetime.time(10, 30, 0)
start_evening_rush = datetime.time(16, 0, 0)
end_evening_rush = datetime.time(18, 0, 0)

In [154]:
# Make rush hour dummies
df_classification_filter['morning_rush_hour_dummy'] = df_classification_filter['vertrekmoment_utc'].apply(lambda x: 1 if start_morning_rush <= x.time() <= end_morning_rush else 0)
df_classification_filter['evening_rush_hour_dummy'] = df_classification_filter['vertrekmoment_utc'].apply(lambda x: 1 if start_evening_rush <= x.time() <= end_evening_rush else 0)

C:\Users\marieke.beerepoo_nsp\AppData\Local\Temp\ipykernel_11072\3533730179.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification_filter['morning_rush_hour_dummy'] = df_classification_filter['vertrekmoment_utc'].apply(lambda x: 1 if start_morning_rush <= x.time() <= end_morning_rush else 0)
C:\Users\marieke.beerepoo_nsp\AppData\Local\Temp\ipykernel_11072\3533730179.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification_filter['evening_rush_hour_dummy'] = df_classification_filt

In [155]:
# Make week/weekend dummy
df_classification_filter['weekend_dummy'] = df_classification_filter['vertrekmoment_utc'].apply(lambda x: 1 if x.weekday()<5 else 0)

C:\Users\marieke.beerepoo_nsp\AppData\Local\Temp\ipykernel_11072\2208506372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification_filter['weekend_dummy'] = df_classification_filter['vertrekmoment_utc'].apply(lambda x: 1 if x.weekday()<5 else 0)


In [156]:
# We want the temperature and rain prediction of a week ago
df_weather_filter = df_weather_filter[df_weather_filter['forecast dt iso'].dt.date == df_weather_filter['slice dt iso'].dt.date - pd.DateOffset(days=7)]
df_weather_filter = df_weather_filter.groupby(df_weather_filter['slice dt iso'].dt.date)['temperature', 'rain'].mean().reset_index()

C:\Users\marieke.beerepoo_nsp\AppData\Local\Temp\ipykernel_11072\503191178.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_weather_filter = df_weather_filter.groupby(df_weather_filter['slice dt iso'].dt.date)['temperature', 'rain'].mean().reset_index()


### Merge dataframes

In [157]:
# Merge the two datafrrames
df_weather_filter['slice dt iso'] = pd.to_datetime(df_weather_filter['slice dt iso'])
merged_df = pd.merge(df_weather_filter, df_classification_filter, left_on='slice dt iso', right_on='verkeersdatum_ams')


## Save preprocessed data

In [160]:
merged_df.to_csv('data/preprocessed_df.csv', index=False)